# Kevin Bethune: CCO/CEO of dreams • design + life

### "We should leverage uncertainty for the powerful, creativity-inducing variable that it is." —via [Reimagining Design: Unlocking Strategic Innovation](https://mitpress.mit.edu/featured-book-reimagining-design/)

### With Kevin, we use embedding models to collect relevant quotes from his past to guide how career advice may happen.

# 🧑‍🍳 Recipe for AI-driven Career Advising

- ~500 tokens from COMPLETION Pre-trained Foundation Model
- ~1000 tokens from EMBEDDING Pre-trained Foundation Model
- 1 list of Reimagining Design quotes
- 1 PLUGIN for career advice
 
First, run the list of quotes through the EMBEDDING model. 

Formulate which expertise you want to draw upon from the list of quotes.

Lastly, take that context and have it drive the Career Advice plugin.

Taste for quality, and then plate appropriately.

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Gather the core tools 🧰

In [35]:
// Load some helper functions, e.g. to load values from settings.json
#!import ../config/Settings.cs 

In [36]:
#r "nuget: Microsoft.SemanticKernel, 0.20.230821.4-preview"
#r "nuget: System.Linq.Async, 6.0.1"

Installed Packages Microsoft.SemanticKernel, 0.20.230821.4-preview System.Linq.Async, 6.0.1

# Fire up a kernel with TWO models 🔥

In [37]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var builder = new KernelBuilder();

if (useAzureOpenAI)
{
    builder.WithAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
}
else
{
    builder.WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);
}

builder.WithMemoryStorage(new VolatileMemoryStore());

var kernel = builder.Build();

# Generate embeddings for later 🧲 gathering

In [38]:
const string MemoryCollectionName = "kevinQuotes";

// Kevin quoted by his fans on GoodReads: https://www.goodreads.com/author/quotes/21599683.Kevin_G_Bethune

await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign1", 
    text: "I believe it’s very important to be more up-front about the hard realities we will face as innovators and still push ourselves forward with a sense of optimism and courage.");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign2", 
    text: "The team needs to learn how to work together, align around the right imperatives, and develop their unique chemistry.");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign3", 
    text: "The constellation of insights must line up in a compelling way, and usually those insights need to come from different vantage points to ensure an idea has a shot of making it into a well-timed, successful innovation that is desirable, viable, and technically feasible.");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign4", 
    text: "We should leverage uncertainty for the powerful, creativity-inducing variable that it is.");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign5", 
    text: "The quest for innovation against newly emerging realities requires us to bring disparate teams closer together and to create more bandwidth for actually collaborating with one another.");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign6", 
    text: "Design still fights the perception of being the very last step in the value stream.");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign7", 
    text: "Thanks to design thinking, most organizations are beginning to see the value of embracing design earlier in their processes.");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign8", 
    text: "Moving on to incubation, as we build out our designed experiences, the designer should no longer (and can no longer) work in isolation from other disciplines.");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign9", 
    text: "Design will play an important role in stringing the collective team’s efforts together in a cohesive way to optimize impact in the marketplace.");

# Experiment 🧪 with what you think is relevant and important 

In [40]:
var questions = new[]
{
    "how to make a pizza",
    "challenges of businesses"
};

foreach (var q in questions)
{
    var response = await kernel.Memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine($"{q}\n> {response?.Metadata.Text}");
}

how to make a pizza
> The team needs to learn how to work together, align around the right imperatives, and develop their unique chemistry.
challenges of businesses
> I believe it’s very important to be more up-front about the hard realities we will face as innovators and still push ourselves forward with a sense of optimism and courage.


# Plate the partially finished meal 🍽️

In [41]:
using Microsoft.AspNetCore.Html;
using Markdig;

String foundInMemory = "";

foreach (var q in questions)
{
    var response = await kernel.Memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
   foundInMemory += $"## {q}\n> {response?.Metadata.Text}\n\n";
}

var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();
var myResult2HTML = Markdown.ToHtml(foundInMemory, myPipeline);
var myHTMLContent = new HtmlContentBuilder();

myHTMLContent.AppendHtml(myResult2HTML);
myHTMLContent

how to make a pizza 
 
 The team needs to learn how to work together, align around the right imperatives, and develop their unique chemistry. 
 
 challenges of businesses 
 
 I believe it’s very important to be more up-front about the hard realities we will face as innovators and still push ourselves forward with a sense of optimism and courage.

# Now let's use a ✨ COMPLETION model that's grounded

In [42]:
var pluginsSKDirectory = "plugins-sk";

var mnFunctions = kernel.ImportSemanticSkillFromDirectory(pluginsSKDirectory, "Mentor");

var result = await mnFunctions["CareerAdviceGeneric"].InvokeAsync(
@"
What should I become?");
Console.WriteLine(result);


As a mentor in the field of design, I would advise you to consider becoming a graphic designer, industrial designer, or user experience (UX) designer. These fields offer a wide range of opportunities to express your creativity and problem-solving skills. Research each area to determine which aligns best with your interests and strengths.


In [43]:
using Microsoft.SemanticKernel.Orchestration;

var myContext = new ContextVariables(); 

myContext.Set("input", "What should I become?"); 
myContext.Set("beliefs", "I believe it’s very important to be more up-front about the hard realities we will face as innovators and still push ourselves forward with a sense of optimism and courage."); 

var myResult = await kernel.RunAsync(myContext,mnFunctions["CareerAdviceGrounded"]);

Console.WriteLine(myResult);


As a mentor in the field of design, I believe it's important to be more up-front about the hard realities we will face as innovators and still push ourselves forward with a sense of optimism and courage.


# Let's look at the science 📐

In [44]:
using System.Text.Json;

var usageresult = result.ModelResults.Select(result => result.GetOpenAIChatResult()?.Usage);
var injson = JsonSerializer.Serialize(usageresult, new JsonSerializerOptions { WriteIndented = true });
Console.WriteLine($"🧱 The MODEL used:\n\n> {model}\n");
Console.WriteLine($"💸 How many TOKENS were used?\n\n{injson}\n");
String functionName = "CareerAdviceGrounded";
Console.WriteLine($"🌡️ What TEMPERATURE setting was used?\n\n> {mnFunctions[functionName].RequestSettings.Temperature} (0 is 🥶 ; 1 is 🥵)\n");
Console.WriteLine($"🙊 What TOP P setting was used?\n\n> {mnFunctions[functionName].RequestSettings.TopP} (0 is 😑 ; 1 is 🤯)");

🧱 The MODEL used:

> gpt-3.5-turbo

💸 How many TOKENS were used?

[
  {
    "CompletionTokens": 63,
    "PromptTokens": 44,
    "TotalTokens": 107
  }
]

🌡️ What TEMPERATURE setting was used?

> 0.3 (0 is 🥶 ; 1 is 🥵)

🙊 What TOP P setting was used?

> 0.5 (0 is 😑 ; 1 is 🤯)
